In [ ]:
import backtrader as bt
import backtrader.feeds as btfeeds

import datetime
import pandas as pd

import schwabdev

import datetime
from dotenv import load_dotenv
import os

In [ ]:
class crossover(bt.Strategy):
    params =(

    )

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print(f"{dt.isoformat()} | {txt}") 

    def __init__(self):
        # close price
        # buy / sell order
        # commission
        # buy price

        # SMA
        # EMA
        # crossover
        pass